In [3]:
import os
import pylorentz
from pylorentz import Momentum4
import numpy as np
# !pip install --user --prefix= pylorentz

In [13]:
def generate_sim_script_filereader(events_properties_filename, script_path=None):
    if script_path is None:
        script_path = os.path.splitext(events_properties_filename)[0]+".mac"
        
    with open(events_properties_filename, 'r') as file:
        nlines_read=0
        found_nevents=False
        while (not found_nevents) and nlines_read<=10:
            line_content = file.readline().split()
            nlines_read+=1
            if "nevents" in line_content:
                try:
                    nevents = int(line_content[-1])
                    found_nevents = True
                    break
                except:
                    print("Could not read number of events")
            

    script = "/det/select Box \n"
    script+= "/gen/select file_reader \n"
    script+= f"/gen/file_reader/pathname {events_properties_filename}\n"
    script+= f"/run/beamOn {nevents}"

    with open(script_path, 'w') as file:
        file.write(script)
        
    print("Script saved at", script_path)
    
    return script_path

def frame_transform(e, p_x, p_y, p_z, boost_direction, beta=None, gamma=None):
    """
    Transform the momentum four-vector with boost_direction and beta/gamma
    """
    x, y, z = boost_direction
    four_momentum = pylorentz.Momentum4(e, p_x, p_y, p_z).boost(x, y, z, beta=beta, gamma=gamma)
    return four_momentum
    
    
def twobody_decay(mass, three_momentum, decayproduct_pid=[13,13], rand_seed=None):
    """
    mass: mass of the decay particle [GeV]
    three_momentum: NON_RELATIVISTIC three momentum of the decay partile [GeV]
    rand_seed: If None, then fresh, unpredictable entropy will be pulled from the OS. 
    """
    mass_map={13: 0.1057, # mu-
             11:0.000511, # e-
             }
    
    m1=mass_map[decayproduct_pid[0]]
    m2=mass_map[decayproduct_pid[1]]
    M=mass
    P=np.sqrt(np.sum(np.power(three_momentum,2)))
    
    # First, 0->1+2 decay in the rest frame of 0:
    E1 = (M**2+m1**2-m2**2)/(2*M)
    E2 = (M**2+m2**2-m1**2)/(2*M)
    p1 = ((M**2+m1**2-m2**2)**2- 4*M**2*m1**2)**0.5/(2*M)
    p2 = ((M**2+m2**2-m1**2)**2- 4*M**2*m2**2)**0.5/(2*M)
    
    # Generate a randomized direction
    rng = np.random.default_rng(seed=rand_seed)
    p1_x = rng.normal(0,1)
    p1_y = rng.normal(0,1)
    p1_z = rng.normal(0,1)
    mag = np.sqrt(p1_x**2+p1_y**2+p1_z**2)
    p1_x = p1_x/mag*p1
    p1_y = p1_y/mag*p1
    p1_z = p1_z/mag*p1
    
    # Boost the 4-momentum
    boost_direction = three_momentum
    boost_gamma = M**2+
    
    

In [31]:
rng = np.random.default_rng(seed=1)

In [32]:
rng.normal(0,1)

0.345584192064786

In [27]:
frame_transform(100, 100,0,0, (1,0,0), 0.99)

Momentum4(7.08881, 7.08881, 0, 0)

In [12]:
generate_sim_script_filereader("/project/def-mdiamond/tomren/jupyter/tracker_6layers/scripts/particle_parameters.tsv")

Script saved at /project/def-mdiamond/tomren/jupyter/tracker_6layers/scripts/particle_parameters.mac


# Combine root files

In [16]:
import ROOT as root
from array import array
from tqdm import tqdm

In [14]:
filename_ins=["/project/def-mdiamond/tomren/mathusla/data/fit_study_6layer/muon_100_GeV/20230503/170343/run0.root",
             "/project/def-mdiamond/tomren/mathusla/data/fit_study_6layer/muon_10_GeV/20230503/170209/run0.root"]
filename_combined = "data/combined.root"
tree_name = "box_run"
nfiles = len(filename_ins)

In [21]:
keys = ['NumHits', 'Hit_energy', 'Hit_time', 'Hit_detId', 'Hit_particlePdgId', 'Hit_G4TrackId', 'Hit_G4ParentTrackId', 'Hit_x', 'Hit_y', 'Hit_z', 'Hit_particleEnergy', 'Hit_particlePx', 'Hit_particlePy', 'Hit_particlePz', 'Hit_weight']
keys_nonused=['NumGenParticles', 'GenParticle_index', 'GenParticle_G4index', 'GenParticle_pdgid', 'GenParticle_status', 'GenParticle_time', 'GenParticle_x', 'GenParticle_y', 'GenParticle_z', 'GenParticle_energy', 'GenParticle_px', 'GenParticle_py', 'GenParticle_pz', 'GenParticle_mo1', 'GenParticle_mo2', 'GenParticle_dau1', 'GenParticle_dau2', 'GenParticle_mass', 'GenParticle_pt', 'GenParticle_eta', 'GenParticle_phi', 'COSMIC_EVENT_ID', 'COSMIC_CORE_X', 'COSMIC_CORE_Y', 'COSMIC_GEN_PRIMARY_ENERGY', 'COSMIC_GEN_THETA', 'COSMIC_GEN_PHI', 'COSMIC_GEN_FIRST_HEIGHT', 'COSMIC_GEN_ELECTRON_COUNT', 'COSMIC_GEN_MUON_COUNT', 'COSMIC_GEN_HADRON_COUNT', 'COSMIC_GEN_PRIMARY_ID', 'EXTRA_11', 'EXTRA_12', 'EXTRA_13', 'EXTRA_14', 'EXTRA_15']


# Open the first file
f0=root.TFile(filename_ins[0])
Tree0 = f0.Get(tree_name)
Nevents = int(Tree0.GetEntries())

# Open the target file
f=root.TFile(filename_combined,'RECREATE')
f.cd()
t_fill=root.TTree(tree_name, tree_name)

# Basic type branch (float) - use array of length 1
NumHits = array('d', [0])
t_fill.Branch('NumHits', NumHits, 'NumHits/D')
# std::vector branch
for key in keys[1:]:
    exec(f"{key} = root.std.vector('double')(300*[0])")
    exec(f"t_fill.Branch(key, {key})")
    
for key in keys_nonused:
    exec(f"{key} = root.std.vector('double')(1*[0])")
    exec(f"t_fill.Branch(key, {key})")    

for ifile in range(nfiles):
    if ifile==0:
        for ii in tqdm(range(Nevents)):
            Tree0.GetEntry(ii)
            
            NumHits[0]=Tree0.NumHits
            for key in keys[1:]:
                # exec(f"temp = [Tree0.{key}[j] for j in range(int(NumHits[0]))]")
                # for j in range(int(NumHits[0])):
                #     exec(f"{key}[j] = Tree0.{key}[j]")
            # t_fill.Fill()
            
    else:
        f1=root.TFile(filename_ins[ifile])
        Tree1 = f1.Get(tree_name)    
        for ii in tqdm(range(Nevents)):
            Tree1.GetEntry(ii)
            
            for key in keys[1:]:
                for j in range(int(Tree1.NumHits)):
                    exec(f"{key}[j+int(t_fill.NumHits)] = Tree1.{key}[j]") 
            NumHits[0]=t_fill.NumHits+Tree1.NumHits
            t_fill.Fill()            
            
            
t_fill.Write()

f.Write()
f.Close()

  2%|▏         | 735/40000 [00:16<14:15, 45.88it/s]Exception ignored in: <generator object tqdm.__iter__ at 0x2b719bb62dd0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/tqdm/std.py", line 1210, in __iter__
    self.close()
  File "/usr/local/lib/python3.8/dist-packages/tqdm/std.py", line 1316, in close
    self.display(pos=0)
  File "/usr/local/lib/python3.8/dist-packages/tqdm/std.py", line 1509, in display
    self.sp(self.__str__() if msg is None else msg)
  File "/usr/local/lib/python3.8/dist-packages/tqdm/std.py", line 350, in print_status
    fp_write('\r' + s + (' ' * max(last_len[0] - len_s, 0)))
  File "/usr/local/lib/python3.8/dist-packages/tqdm/std.py", line 343, in fp_write
    fp.write(_unicode(s))
  File "/usr/local/lib/python3.8/dist-packages/tqdm/utils.py", line 145, in inner
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/ipykernel/iostream.py", line 531, in write
    self.pub_thread.schedule(lambda: s

KeyboardInterrupt: 